In [1]:
import glob
import os
import numpy as np
import math
from DataProcess import *
from SolutionChecker import *
from LocalSearch import *
import cProfile

In [2]:
data_path = os.path.dirname(os.path.abspath('__file__')) + "/data/wildfire.data"

In [3]:
solution_path = os.path.dirname(os.path.abspath('__file__')) + "/data/wildfire.solution"

In [4]:
print_data(data_path)

Add evacuation info
Add graph info
-----------------------------
evacuation_tree =  [[1, 48, 8, 3, 4, 5, 6], [2, 30, 5, 3, 4, 5, 6], [3, 33, 3, 2, 5, 6]]
graph =  [[1, 4, 9223372036854775807, 7, 8], [2, 4, 9223372036854775807, 4, 5], [3, 5, 9223372036854775807, 6, 3], [4, 5, 9223372036854775807, 9, 10], [5, 6, 9223372036854775807, 12, 11]]
number of evacuation nodes :  3
List of evac nodes :  [1, 2, 3]
number of nodes =  6
number of edges =  5
-----------------------------


0

In [5]:
print_solution(solution_path)

-----------------------------
A solution is : [[1, 8, 3], [2, 5, 0], [3, 3, 0]]
f_objectif =  37
Nature of solution :  valid
-----------------------------


0

In [6]:
verify_solution(data_path,solution_path)

Add evacuation info
Add graph info
Add evacuation info
Add graph info
-----------------------------
evacuation_tree =  [[1, 48, 8, 3, 4, 5, 6], [2, 30, 5, 3, 4, 5, 6], [3, 33, 3, 2, 5, 6]]
graph =  [[1, 4, 9223372036854775807, 7, 8], [2, 4, 9223372036854775807, 4, 5], [3, 5, 9223372036854775807, 6, 3], [4, 5, 9223372036854775807, 9, 10], [5, 6, 9223372036854775807, 12, 11]]
number of evacuation nodes :  3
List of evac nodes :  [1, 2, 3]
number of nodes =  6
number of edges =  5
-----------------------------
-----------------------------
A solution is : [[1, 8, 3], [2, 5, 0], [3, 3, 0]]
f_objectif =  37
Nature of solution :  valid
-----------------------------
CREATION TASKS & VERIFY CONSTRAINTS
-----------------------------
ressources info after evacuation =  {'Cap of edge[1-4]': array([8, 8, 8, 0, 0, 0, 0, 0, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]), 'Cap of edge[2-4]': array([0, 0, 0, 0, 0, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5

True

In [7]:
EVA_TREE,GRAPH,NB_NODES = read_data(data_path)
NB_EVA_NODES = len(EVA_TREE)
NB_EDGES = len(GRAPH)
LIST_EVA_NODES = [item[0] for item in EVA_TREE]

Add evacuation info
Add graph info


In [8]:
# for i in range(NB_EVA_NODES) : 
#     node_id = EVA_TREE[i][0]
#     print('Evacuation of (only) node ',node_id,' is ', get_duration(node_id,EVA_TREE,GRAPH),' mins')

In [9]:
print('Borne inferieur = {} mins'.format(get_borne_inf(LIST_EVA_NODES,EVA_TREE,GRAPH)))

Borne inferieur = 34 mins


In [10]:
print('Borne superieur = {} mins'.format(get_borne_sup(LIST_EVA_NODES,EVA_TREE,GRAPH)[0]))

Borne superieur = 40 mins


In [11]:
data_path_2 = os.path.dirname(os.path.abspath('__file__')) + "/data/dense.data"
EVA_TREE,GRAPH,NB_NODES = read_data(data_path_2)
NB_EVA_NODES = len(EVA_TREE)
NB_EDGES = len(GRAPH)
LIST_EVA_NODES = [item[0] for item in EVA_TREE]

Add evacuation info
Add graph info


In [12]:
print_data(data_path_2)

Add evacuation info
Add graph info
-----------------------------
evacuation_tree =  [[227, 2483, 71, 19, 237, 244, 57, 529, 548, 70, 87, 111, 503, 382, 531, 225, 481, 489, 493, 437, 385, 346, 373], [344, 157, 72, 25, 221, 257, 264, 162, 77, 3, 19, 29, 57, 529, 548, 70, 87, 111, 503, 382, 531, 225, 481, 489, 493, 437, 385, 346, 373], [323, 986, 71, 20, 333, 337, 342, 498, 515, 520, 209, 216, 473, 161, 166, 170, 211, 250, 281, 310, 328, 336, 346, 373], [268, 2792, 132, 11, 230, 486, 170, 211, 250, 281, 310, 328, 336, 346, 373], [275, 1707, 70, 23, 113, 372, 78, 83, 337, 342, 498, 515, 520, 209, 216, 473, 161, 166, 170, 211, 250, 281, 310, 328, 336, 346, 373], [279, 2090, 72, 38, 179, 92, 110, 124, 187, 140, 484, 158, 109, 118, 160, 42, 51, 518, 523, 528, 537, 545, 3, 19, 29, 57, 529, 548, 70, 87, 111, 503, 382, 531, 225, 481, 489, 493, 437, 385, 346, 373], [359, 1155, 70, 52, 189, 410, 456, 458, 56, 58, 202, 205, 208, 519, 16, 90, 98, 114, 132, 55, 20, 527, 421, 332, 247, 190, 142, 61, 5

0

In [13]:
print('Borne inferieur = {} mins'.format(get_borne_inf(LIST_EVA_NODES,EVA_TREE,GRAPH)))

Borne inferieur = 91 mins


In [14]:
print('Borne superieur = {} mins'.format(get_borne_sup(LIST_EVA_NODES,EVA_TREE,GRAPH)[0]))

Borne superieur = 132 mins


In [15]:
init_sol = get_borne_sup(LIST_EVA_NODES,EVA_TREE,GRAPH)[1]
init_sol

[[279, 71, 30],
 [279, 71, 30],
 [268, 132, 0],
 [275, 70, 67],
 [275, 70, 67],
 [323, 71, 208],
 [163, 70, 85],
 [384, 71, 213],
 [384, 71, 213],
 [323, 71, 208]]

In [16]:
cp = cProfile.Profile()
cp.enable()
best_solution,endtime = LocalSearchRun(init_sol,EVA_TREE,GRAPH,n_iter=20)
cp.disable()
cp.print_stats()

[268, 279, 279, 275, 275, 163, 323, 323, 384, 384]  =>  165
[268 279 279 323 275 163 275 323 384 384]  =>  162
[268 384 279 323 275 163 275 323 279 384]  =>  140
         9775390 function calls (9775378 primitive calls) in 65.109 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   65.109   65.109 <ipython-input-16-f58701e0b131>:3(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-16-f58701e0b131>:4(<module>)
     9030    0.156    0.000   11.729    0.001 DataProcess.py:102(get_task)
     9030    0.271    0.000   10.992    0.001 DataProcess.py:106(<listcomp>)
      903    0.041    0.000    3.263    0.004 DataProcess.py:121(create_solution)
     9030    0.735    0.000    3.218    0.000 DataProcess.py:124(<listcomp>)
      903    7.674    0.008   65.020    0.072 DataProcess.py:143(get_end_time)
   267129   22.337    0.000   22.337    0.000 DataProcess.py:172(<listcomp>)
      903   

In [17]:
print('solution = ',best_solution)
print('end time = ',endtime)

solution =  [[268, 132, 0], [384, 71, 101], [279, 71, 60], [323, 71, 158], [275, 70, 149], [163, 70, 60], [275, 70, 149], [323, 71, 158], [279, 71, 60], [384, 71, 101]]
end time =  140


In [18]:
# #-----------------------------------------------------------------------------
# # Build the model
# #-----------------------------------------------------------------------------
# from docplex.cp.model import CpoModel
# from docplex.cp.expression import *
# from docplex.cp.solution import *
# import docplex.cp.utils_visu as visu
# # Create model
# mdl = CpoModel(version='12.7.0')

In [19]:
# # Create one interval variable per job operation
# E_nodes = [[mdl.interval_var(size=get_task(j,EVA_TREE,GRAPH)[0], name="Evacuees from {} at {}".format(j,m)) for m in get_task(j,EVA_TREE,GRAPH)[1]] for j in LIST_EVA_NODES]
# len(E_nodes)

In [20]:
# len(E_nodes[0])

In [21]:
# #-----------------------------------------------------------------------------
# # Constraints 
# #-----------------------------------------------------------------------------

# #Each operation must start after a precise duration = time to traverse an arc
# for i in range(NB_EVA_NODES) :
#     root = LIST_EVA_NODES[i]
#     route_list = get_task(root,EVA_TREE,GRAPH)[1]
#     route_len = len(route_list)
#     current = root
#     for m in range(route_len) :
#         nxt = route_list[m]
# #         print("Node-{}-to-{}".format(root,nxt))
#         if nxt != current :
# #             print(current,nxt)
#             _,delay,_ = get_edge_info(current,nxt,GRAPH)
#             mdl.add(mdl.start_at_start(E_nodes[i][m-1],E_nodes[i][m],delay))
#             print("Evacuees-from-{}-at-{} starts after Evacuees-from-{}-at-{} : {} mins ".format(root,nxt,root,current,delay))
#         current = nxt
#     print('----------')

In [22]:
# #Constraint on capacity of ressources, which are edges of graph

# for r in range(NB_EDGES) :
#     edge = GRAPH[r]
# #     print(edge)
#     CAP_MAX = edge[-1]
#     rsc = 0
#     ressources = []
#     for i in range(NB_EVA_NODES) :
#         root = LIST_EVA_NODES[i]
#         _,route_list,capacity = get_task(root,EVA_TREE,GRAPH)
#         route_len = len(route_list)
#         current = root

#         for m in range(route_len) : 
#             nxt = route_list[m]

#             if edge[0] == current and edge[1]==nxt :
#                 ressources.append(mdl.pulse(E_nodes[i][m-1],capacity))
#                 print("Evacuees-from-{}-at-{} requests edge [{}-{}] a capacity {}".format(root,route_list[m-1],current,nxt,capacity))
                
#                 rsc += capacity
               
#             current = nxt
#         #end for
#     #end for
#     mdl.add(mdl.sum(ressources) <= CAP_MAX)
#     print('Constraint on edge {}-{} with capacity_max {}... is loaded'.format(edge[0],edge[1],CAP_MAX))
#     print('--------------')
# #end for

In [23]:
# #Minimize end of all tasks
# mdl.add(mdl.minimize(mdl.max([mdl.end_of(node) for list_nodes in E_nodes for node in list_nodes])))

In [24]:
# #-----------------------------------------------------------------------------
# # Solve the model and display the result
# #-----------------------------------------------------------------------------
# mdl.write_information()

In [25]:
# # Solve model
# # print("Solving model....")
# msol = mdl.solve(TimeLimit=1000)
# # print("Solution: ")
# msol.print_solution()